In [1]:
import pandas as pd
import numpy as np
import spacy
import os
import csv
import sys
import json
# df = pd.read_csv('tech_file.csv')
# orig_txt = df['transcript'][25]
import torch
import random
from evaluate import load

random.seed(1234)
np.random.seed(1234)
torch.manual_seed(1234)


/home/bizon/anaconda3/envs/w210_podcast_ollama/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

bertscore = load("bertscore")
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]
results = bertscore.compute(predictions=predictions, references=references, lang="en")

tokenizer_config.json: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25.0/25.0 [00:00<00:00, 66.1kB/s]
config.json: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 482/482 [00:00<00:00, 1.71MB/s]
vocab.json: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 899k/899k [00:00<00:00, 1.37MB/s]
merges.txt: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 456k/456k [00:00<00:00, 814kB/s]
model.safetensors: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1.42G/1.42G [21:33<00:00, 1.10MB/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: 

In [4]:
results

{'precision': [0.9999999403953552, 1.0],
 'recall': [0.9999999403953552, 1.0],
 'f1': [0.9999999403953552, 1.0],
 'hashcode': 'roberta-large_L17_no-idf_version=0.3.9(hug_trans=4.31.0)'}

In [5]:
csv.field_size_limit(sys.maxsize)

podcast_data = []
row_num = 0
with open('../summarize/vtt_data.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter='|')
    for row in reader:
        row_num += 1
        
        if row_num == 1:
            continue
            
        filename = row[5]
        if not filename.endswith("_large.vtt"):
            continue

        podcast = {    
            "episode_index": row[0],    
            "guest": row[1],
            "episode_name": row[2],
            "host_name": row[3],
            "episode_number": row[4],
            "transcript": row[6],
            "duration": row[7],
        }
        podcast_data.append(podcast)

In [6]:
benchmark_summaries = {
    "12" : "../summarize/summarized_dataset/podcast_summaries_openai_gpt35turbo_12_benchmark.json",
    "22" : "../summarize/summarized_dataset/podcast_summaries_openai_gpt35turbo_22_benchmark.json",
    "23" : "../summarize/summarized_dataset/podcast_summaries_openai_gpt35turbo_23_benchmark.json",
    "94" : "../summarize/summarized_dataset/podcast_summaries_openai_gpt35turbo_94_benchmark.json",
}

keypoints_summaries = {
    "12" : "../summarize/summarized_dataset/podcast_summaries_openai_gpt35turbo_12_stage3_extractkeypoints_v2b4.json",
    "22" : "../summarize/summarized_dataset/podcast_summaries_openai_gpt35turbo_22_stage3_extractkeypoints_v2b4.json",
    "23" : "../summarize/summarized_dataset/podcast_summaries_openai_gpt35turbo_23_stage3_extractkeypoints_v2b4.json",
    "94" : "../summarize/summarized_dataset/podcast_summaries_openai_gpt35turbo_94_stage3_extractkeypoints_v2b4_ep94only.json",
}

In [7]:

transcripts = []
benchmarks = []
keypoints = []

count = 0
for podcast in podcast_data:

    if int(podcast['episode_number']) != 94 and int(podcast['episode_number']) != 23 and \
       int(podcast['episode_number']) != 12 and int(podcast['episode_number']) != 22:
#         print(f"episode {podcast['episode_number']} already processed. skip")
        continue
    
    transcripts.append(podcast['transcript'])
    
    print(f"Calculating score for episode: {podcast['episode_number']}")    
    summary = benchmark_summaries[podcast['episode_number']] 
    
    summary_json = None
    with open(summary) as f:
        summary_json = json.load(f)
    
    benchmarks.append(summary_json['final_summary'])
    
    # keypoints
    summary = keypoints_summaries[podcast['episode_number']] 
    
    summary_json = None
    with open(summary) as f:
        summary_json = json.load(f)
    
    keypoints.append(summary_json['final_summary'])

    
benchmark_results = bertscore.compute(predictions=benchmarks, references=transcripts, lang="en")
    
print("benchmark_results:")
print(benchmark_results)

keypoint_results = bertscore.compute(predictions=keypoints, references=transcripts, lang="en")

print("keypoint_results:")
print(keypoint_results)
    

Calculating score for episode: 12
Calculating score for episode: 22
Calculating score for episode: 23
Calculating score for episode: 94
benchmark_results:
{'precision': [0.8271299600601196, 0.8481485843658447, 0.830432653427124, 0.818525493144989], 'recall': [0.8189882040023804, 0.8240691423416138, 0.8178576231002808, 0.8038623332977295], 'f1': [0.8230389952659607, 0.8359354734420776, 0.8240971565246582, 0.8111276030540466], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.9(hug_trans=4.31.0)'}
keypoint_results:
{'precision': [0.8322412967681885, 0.8500759601593018, 0.8304056525230408, 0.8181145191192627], 'recall': [0.8268346786499023, 0.8384329080581665, 0.8199012875556946, 0.8053003549575806], 'f1': [0.8295291662216187, 0.8442143201828003, 0.8251200914382935, 0.8116568326950073], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.9(hug_trans=4.31.0)'}


In [11]:

benchmark_results_avg_precision = sum(benchmark_results['precision'])/len(benchmark_results['precision'])
benchmark_results_avg_recall = sum(benchmark_results['recall'])/len(benchmark_results['recall'])

print(f"NE benchmark_results_avg_precision: {benchmark_results_avg_precision:.4f}")
print(f"NE benchmark_results_avg_recall: {benchmark_results_avg_recall:.4f}")

keypoint_results_avg_precision = sum(keypoint_results['precision'])/len(keypoint_results['precision'])
keypoint_results_avg_recall = sum(keypoint_results['recall'])/len(keypoint_results['recall'])

print(f"NE keypoint_results_avg_precision: {keypoint_results_avg_precision:.4f}")
print(f"NE keypoint_results_avg_recall: {keypoint_results_avg_recall:.4f}")

NE benchmark_results_avg_precision: 0.8311
NE benchmark_results_avg_recall: 0.8162
NE keypoint_results_avg_precision: 0.8327
NE keypoint_results_avg_recall: 0.8226
